In [9]:
import json
import networkx as nx
import random
import matplotlib.pyplot as plt
import math
log = math.log2
from sympy import *
from numpy.polynomial import Polynomial as P


In [38]:
def import_channel_graph():
    # retrieve this by: lightning-cli listchannels > listchannels.json
    f = open("listchannels.json")
    jsn = json.load(f)
    G = nx.Graph()
    for channel in jsn["channels"]:
        src = channel["source"]
        dest = channel["destination"]
        cap = int(int(channel["satoshis"])/1000)
        sid = channel["short_channel_id"]
        G.add_edge(src,dest,weight=cap,sid=sid)
    return G

channel_graph = import_channel_graph()

In [11]:
def create_probability_graph(amt, channel_graph):
    prob_graph = nx.DiGraph()
    for src, dest in channel_graph.edges():
        cap = channel_graph[src][dest]["weight"]
        if cap <amt:
            continue
        success_probability = (cap - amt + 1) / (cap + 1)
        log_prob = log(success_probability)
        weight = -1*log_prob 
        #TODO: question could we ignore the amt and use a weight that is based solely on capacities?
        prob_graph.add_edge(src,dest,weight=weight)
        prob_graph.add_edge(dest,src,weight=weight)
        
    return prob_graph

In [12]:
def next_hop(path):
    for i in range(1,len(path)):
        src = path[i-1]
        dest = path[i]
        yield (src,dest)
        
def print_path(path,network):
    string = ""
    for src,dest in next_hop(path):
        string+=src[:10]
        string+="--"+str(network[src][dest]["weight"])+"-->"
    string+=dest[:10]
    print(string)
    
def path_weight(path,network):
    return sum(network[src][dest]["weight"] for src, dest in next_hop(path))

In [6]:
def uni_probability(capacities,x):
    eq = 1
    for cap in capacities:
        eq=eq*((cap+1-x)/(cap+1))
    return eq

def solve_break_even(path1, path2):
    #print(path1, path2)
    x = symbols("x")
    caps = [int(channel_graph[src][dest]["weight"]) for src, dest in next_hop(path1)]
    p2 = 2**(-1*path_weight(path2,prob_graph))
    #print(p2)
    t1 = uni_probability(caps,x)
    c2 = [c-1 for c in caps]
    t2 = uni_probability(c2,x)
    print("solution of: ",t1*p2, "=", t2)
    #print(t2)
    test = Poly(t1*p2-t2)
    sols = [N(sol) for sol in test.all_roots() if N(sol)>0]
    split = min(sols)
    return split

In [7]:

for i in range(4):
    top_k_paths = []
    print("PAYMENT PAIR NUMBER: {}".format(i))
    n = [x for x in channel_graph.nodes() if len(list(channel_graph.neighbors(x)))>10]
    print(len(n), "nodes to sample from")
    payment_pair = random.sample(n,2)
    src, dest = payment_pair
    
    prob_graph = create_probability_graph(100,channel_graph)
    print("to: ", dest)
    paths=nx.shortest_simple_paths(prob_graph,"03efccf2c383d7bf340da9a3f02e2c23104a0e4fe8ac1a880c8e2dc92fbdacd9df",dest,weight="weight")
    
    k =3
    top_k_paths = []
    for i, path in enumerate(paths):
        if i>1:
            break
        print("path numer: {}, logprob: {:6.4f} prob: {:6.4f}".format(i, path_weight(path,prob_graph), 2**(-1*path_weight(path, prob_graph))))
        #print_path(path,prob_graph)
        print_path(path,channel_graph)
        print("")
        top_k_paths.append(path)
    split = solve_break_even(top_k_paths[0], top_k_paths[1])
    print("split value: {:6}\n\n\n".format(int(split)))

PAYMENT PAIR NUMBER: 0
891 nodes to sample from
to:  02354d23deeacfe0fecbfff109950cff14a1d2da72c1eb98a109b7ec0133b06609
path numer: 0, logprob: 0.0809 prob: 0.9455
03efccf2c3--16777.215-->02247d9db0--16777.215-->03fce16553--1500.0-->02354d23de

path numer: 1, logprob: 0.0854 prob: 0.9425
03efccf2c3--10000.0-->03cde60a63--500000.0-->030c3f19d7--500000.0-->03baa70886--16770.0-->03fce16553--1500.0-->02354d23de

solution of:  0.942539397397957*(-x/1501 + 1)*(-x/16778 + 1)**2 = (-x/1500 + 1)*(-x/16777 + 1)**2
split value:   1483



PAYMENT PAIR NUMBER: 1
891 nodes to sample from
to:  038b0e5d711f3112e6b0ffd2b6d0ae90977a0b5eb309f5bfbab89e6d6fd0fd7a8f
path numer: 0, logprob: 0.0307 prob: 0.9789
03efccf2c3--10000.0-->03cde60a63--200000.0-->03864ef025--5000.0-->038b0e5d71

path numer: 1, logprob: 0.0309 prob: 0.9788
03efccf2c3--10000.0-->03cde60a63--500000.0-->030c3f19d7--500000.0-->03baa70886--477184.791-->03864ef025--5000.0-->038b0e5d71

solution of:  0.978838277651289*(-x/5001 + 1)*(-x/10001

# inefficient split finding algorithm

In [62]:
def path_identifier(path):
    return ":".join(p[:7] for p in path)

def update_prob_graph(prob_graph,channel_graph,paths,delta):
    for path, amt in paths: 
        for src, dest in next_hop(path):
            cap = channel_graph[src][dest]["weight"]
            success_probability = (cap - amt +1 -delta) / (cap - amt + 1)
            log_prob = log(success_probability)
            weight = -1*log_prob 
            if success_probability <= 0:
                print("can't updatedate probability")
                print(path,src,dest,success_probability)
                weight = 100000000
            #TODO: question could we ignore the amt and use a weight that is based solely on capacities?
            prob_graph[src][dest]["weight"]=weight
            
def compute_mpp_likelihood(channel_graph, paths):
    sids = {}
    caps = {}
    for path, amt in paths:
        for src, dest in next_hop(path):
            sid = channel_graph[src][dest]["sid"]
            if sid in sids:
                sids[sid]+=amt
            else:
                sids[sid]=amt
            caps[sid]=channel_graph[src][dest]["weight"]
    res = 1
    for sid,amt in sids.items():
        if amt > caps[sid]:
            return 0
        else:
            res*=(caps[sid]-amt+1)/(caps[sid]+1)
    return res

In [63]:
n = [x for x in channel_graph.nodes() if len(list(channel_graph.neighbors(x)))>10]
print(len(n), "nodes to sample from")
destination_nodes = random.sample(n,10)
destination_nodes = ["02768cf079dbbeafc174221d377a6fc46fea6a49ab67deddbd8165f43950a075d3"]
src = "03efccf2c383d7bf340da9a3f02e2c23104a0e4fe8ac1a880c8e2dc92fbdacd9df"

891 nodes to sample from


In [64]:
top_k_paths = {}
for dest in destination_nodes:
    top_k_paths = {}
    print("to: ", dest)
    steps = 20
    amt = 5000
    delta = amt/steps
    prob_graph=create_probability_graph(delta,channel_graph)
    
    for i in range(steps):
        #print(i)
        path = nx.shortest_path(prob_graph,src,dest,weight="weight")
        key = path_identifier(path)
        print(i, "sats asigned 1 additional sat along", key)
        print_path(path, prob_graph)
        if key in top_k_paths:
            top_k_paths[key]=(path,top_k_paths[key][1]+delta)
        else: 
            top_k_paths[key]=(path,delta)
        update_prob_graph(prob_graph,channel_graph,top_k_paths.values(),delta)
        print(len(top_k_paths),"different paths\n")
    print(top_k_paths)
    break


to:  02768cf079dbbeafc174221d377a6fc46fea6a49ab67deddbd8165f43950a075d3
0 sats asigned 1 additional sat along 03efccf:03cde60:03bb88c:02768cf
03efccf2c3--0.0253152441413971-->03cde60a63--0.00501244132304683-->03bb88ccc4--0.133460002297882-->02768cf079
1 different paths

1 sats asigned 1 additional sat along 03efccf:03cde60:03bb88c:02768cf
03efccf2c3--0.0259727876283696-->03cde60a63--0.00503769251431339-->03bb88ccc4--0.154055500652390-->02768cf079
1 different paths

2 sats asigned 1 additional sat along 03efccf:03cde60:03bb88c:02768cf
03efccf2c3--0.0266654024354309-->03cde60a63--0.00506319941029776-->03bb88ccc4--0.182188321166551-->02768cf079
1 different paths

3 sats asigned 1 additional sat along 03efccf:03cde60:03bb88c:02768cf
03efccf2c3--0.0273959715142333-->03cde60a63--0.00508896591484262-->03bb88ccc4--0.222943731151691-->02768cf079
1 different paths

4 sats asigned 1 additional sat along 03efccf:03cde60:03bb88c:02768cf
03efccf2c3--0.0281677027212542-->03cde60a63--0.005114996011664

14 sats asigned 1 additional sat along 03efccf:02247d9:03abf6f:02768cf
03efccf2c3--0.0159717188061888-->02247d9db0--0.0159717188061888-->03abf6f44c---0.305758056376118-->02768cf079
can't updatedate probability
['03efccf2c383d7bf340da9a3f02e2c23104a0e4fe8ac1a880c8e2dc92fbdacd9df', '03cde60a6323f7122d5178255766e38114b4722ede08f7c9e0c5df9b912cc201d6', '03bb88ccc444534da7b5b64b4f7b15e1eccb18e102db0e400d4b9cfe93763aa26d', '02768cf079dbbeafc174221d377a6fc46fea6a49ab67deddbd8165f43950a075d3'] 03bb88ccc444534da7b5b64b4f7b15e1eccb18e102db0e400d4b9cfe93763aa26d 02768cf079dbbeafc174221d377a6fc46fea6a49ab67deddbd8165f43950a075d3 -249.0
can't updatedate probability
['03efccf2c383d7bf340da9a3f02e2c23104a0e4fe8ac1a880c8e2dc92fbdacd9df', '03cde60a6323f7122d5178255766e38114b4722ede08f7c9e0c5df9b912cc201d6', '0288be11d147e1525f7f234f304b094d6627d2c70f3313d7ba3696887b261c4447', '0374eb2306a27dffd657c781065bd11785588f4811c4949070f8aed79e95a97db4', '02768cf079dbbeafc174221d377a6fc46fea6a49ab67deddbd8165f43

In [65]:
print(compute_mpp_likelihood(channel_graph,list(top_k_paths.values())[0:1]))
print(compute_mpp_likelihood(channel_graph,list(top_k_paths.values())[1:2]))
print(compute_mpp_likelihood(channel_graph,list(top_k_paths.values())[2:3]))
print(compute_mpp_likelihood(channel_graph,list(top_k_paths.values())))


0
0.0018311815144199254
0
0


In [50]:
top_k_paths

{'03efccf:02247d9:03abf6f:02768cf': (['03efccf2c383d7bf340da9a3f02e2c23104a0e4fe8ac1a880c8e2dc92fbdacd9df',
   '02247d9db0dfafea745ef8c9e161eb322f73ac3f8858d8730b6fd97254747ce76b',
   '03abf6f44c355dec0d5aa155bdbdd6e0c8fefe318eff402de65c6eb2e1be55dc3e',
   '02768cf079dbbeafc174221d377a6fc46fea6a49ab67deddbd8165f43950a075d3'],
  250.0),
 '03efccf:03cde60:0288be1:0374eb2:02768cf': (['03efccf2c383d7bf340da9a3f02e2c23104a0e4fe8ac1a880c8e2dc92fbdacd9df',
   '03cde60a6323f7122d5178255766e38114b4722ede08f7c9e0c5df9b912cc201d6',
   '0288be11d147e1525f7f234f304b094d6627d2c70f3313d7ba3696887b261c4447',
   '0374eb2306a27dffd657c781065bd11785588f4811c4949070f8aed79e95a97db4',
   '02768cf079dbbeafc174221d377a6fc46fea6a49ab67deddbd8165f43950a075d3'],
  250.0),
 '03efccf:03cde60:03bb88c:02768cf': (['03efccf2c383d7bf340da9a3f02e2c23104a0e4fe8ac1a880c8e2dc92fbdacd9df',
   '03cde60a6323f7122d5178255766e38114b4722ede08f7c9e0c5df9b912cc201d6',
   '03bb88ccc444534da7b5b64b4f7b15e1eccb18e102db0e400d4b9cfe93

# unused code snippets

In [ ]:
# via: https://stackoverflow.com/questions/63259998/minimize-function-with-sympy-variables-in-scipy
from scipy.optimize import minimize
import sympy as sp

x2_symbol = sp.Symbol('x2')
u2_symbol = sp.Symbol('u2')
fm = 25 * u2_symbol - 20.0 * (sp.sin(x2_symbol)) + 38.7296387 * (sp.cos(x2_symbol)) - 38.7296387

def func(x):
    x2_float, u2_float = x
    return -fm.subs([(x2_symbol, x2_float), (u2_symbol, u2_float)])

def constraint1(x):
    x2_float, u2_float = x
    return -u2_float + 40 * sp.sin(x2_float) + 0.2

def constraint2(x):
    x2_float, u2_float = x
    return -u2_float - 40 * sp.sin(x2_float) + 0.2

b = [-1, 1]
bounds = [b, b]
con1 = {'type': 'ineq', 'fun': constraint1}
con2 = {'type': 'ineq', 'fun': constraint2}
constraints = (con1, con2)
x0 = [0, 0]
solution = minimize(func, x0, method='SLSQP', bounds=bounds, constraints=constraints)
print(solution)

In [ ]:
def create_and_solve_lagrange_problem(eqs,variables,amt,mins):
    print(amt,variables,eqs)
    P = 1
    for eq in eqs:
        P=P*eq
    print(P)
    partial_derrivatives = [diff(P,var) for var in variables]
    l = symbols("l")
    non_linear_equations = [Eq(p,l) for p in partial_derrivatives]
    split = 0
    for var in variables:
        split=split+var
    variables.append(l)
    non_linear_equations.append(Eq(split,amt))
    s = sum(mins)
    start = [amt*m/s for m in mins]
    start.append(1)
    print(non_linear_equations,split, start)
    print((P).evalf(subs=dict(zip(variables[:-1],start))))
    res = nsolve(non_linear_equations,variables,start, simplify=False)
    print(amt, res)
    subs=dict(zip(variables[:-1],res[:-1]))
    print((P).evalf(subs = subs))
    return res

In [ ]:
top_k_paths


    #caps = [int(channel_graph[src][dest]["weight"]) for src, dest in next_hop(path2)]
    #t2=uni_probability(caps,x)
    
    #subs=dict(zip(variables[:-1],res[:-1]))
    #print((P).evalf(subs = subs))
    
    for i in range(int(split)-100,int(split)+100,10):
        subs={x:i+1}
        subs2 = {x:i}
        print(i,t1.evalf(subs=subs),(p2*t1).evalf(subs=subs2))

    x = symbols("x",positive=True)
    y = symbols("y",positive=True)
    caps = [int(channel_graph[src][dest]["weight"]) for src, dest in next_hop(path1)]
    t1 = uni_probability(caps,x)
    caps = [int(channel_graph[src][dest]["weight"]) for src, dest in next_hop(path2)]
    t2=uni_probability(caps,y)
    
    res = create_and_solve_lagrange_problem([t1,t2],[x,y],split,[split,0])
    print(res)


In [69]:
for a in ["74","29"]:
    for b in ["tobisik","Tobisik","TobisiK","TobIsiK","ToBiSiK"]:
        date = None    
        if a == "74":
            date = ["29"]
        else: 
            date = ["74","09","04"]
        for c in date:
            for d in ["!", "?"]:
                print(a+b+d+c)

74tobisik!29
74tobisik?29
74Tobisik!29
74Tobisik?29
74TobisiK!29
74TobisiK?29
74TobIsiK!29
74TobIsiK?29
74ToBiSiK!29
74ToBiSiK?29
29tobisik!74
29tobisik?74
29tobisik!09
29tobisik?09
29tobisik!04
29tobisik?04
29Tobisik!74
29Tobisik?74
29Tobisik!09
29Tobisik?09
29Tobisik!04
29Tobisik?04
29TobisiK!74
29TobisiK?74
29TobisiK!09
29TobisiK?09
29TobisiK!04
29TobisiK?04
29TobIsiK!74
29TobIsiK?74
29TobIsiK!09
29TobIsiK?09
29TobIsiK!04
29TobIsiK?04
29ToBiSiK!74
29ToBiSiK?74
29ToBiSiK!09
29ToBiSiK?09
29ToBiSiK!04
29ToBiSiK?04
